In [ ]:
import bilby
import numpy as np
from scipy import signal
import math
import seaborn as sns

import h5py 
from pathlib import Path
import inspect

import matplotlib.pyplot as plt

def mod_sinusoid(time, A_0, A_am, A_fm, omega_0, omega_ax, phi_c, phi_ax):
    
    amplitude = A_0 + A_am * np.cos(1 * omega_ax * time - phi_ax)
    
    omega_c = omega_0 + A_fm * np.sin(1 * omega_ax * time - phi_ax - math.pi)
    
    phase = omega_c * time - phi_c
    
    return amplitude * np.exp(1j * phase) 

class ComplexGaussianLikelihood(bilby.Likelihood):
    def __init__(self, x, y, sigma, function):
        """
        A general Gaussian likelihood - the parameters are inferred from the
        arguments of function

        Parameters
        ----------
        x, y: array_like
            The data to analyze
        sigma: float
            The standard deviation of the noise
        function:
            The python function to fit to the data. Note, this must take the
            dependent variable as its first argument. The other arguments are
            will require a prior and will be sampled over (unless a fixed
            value is given).
        """
        self.x = x
        self.y = y
        self.sigma = sigma
        self.N = len(x)
        self.function = function
        
        parameters = inspect.getargspec(function).args
        parameters.pop(0)
        super().__init__(parameters=dict.fromkeys(parameters))
        
    def log_likelihood(self):
        
        res = (self.y.real - self.function(self.x, **self.parameters).real) ** 2\
        + (self.y.imag - self.function(self.x, **self.parameters).imag) ** 2
        
        return   - 0.5 * self.N * np.log(2*math.pi*self.sigma**2) - 0.5 * np.sum(res) / (self.sigma ** 2)    


injection_parameters = {
    'A_0': 5e-9,
    'A_am': 5e-9,
    'A_fm': 4e4,
    'omega_0': 2 * math.pi * 58.0e6,
    'omega_ax': 2 * math.pi * 25e6,
    'phi_c': 0,
    'phi_ax': 0,
}

priors = {
    'A_0': bilby.core.prior.Uniform(4.0e-9, 6.0e-9, 'A_0'),
    'A_am': bilby.core.prior.Uniform(4.0e-9, 6.0e-9, 'A_am'),
    'A_fm': bilby.core.prior.Uniform(3.0e4, 5.0e4, 'A_fm'),
    'omega_0': 2 * math.pi * 58.0e6,
    'omega_ax': 2 * math.pi * 25e6,
    'phi_c': 0,
    'phi_ax': 0, 
}

f_sample = 205e6
n_sample = 3 * 8192
time = np.arange(0, n_sample, 1) / f_sample

noise_var = 1.38e-23 * f_sample * 50 * 5e-4

rng = np.random.default_rng()

noise = np.random.multivariate_normal([0, 0], np.eye(2) * noise_var / 2, n_sample)
noise = noise[:, 0] + 1j * noise[:, 1]

signal = mod_sinusoid(time, **injection_parameters) + noise

likelihood = ComplexGaussianLikelihood(time, signal, np.sqrt(noise_var)/2, mod_sinusoid)

result = bilby.run_sampler(
    likelihood = likelihood,
    priors=priors,
    sampler='dynesty',
    npoints=256,
    injection_parameters=injection_parameters,
    outdir='outdir_mod_sine_fast',
    label='1',
)

result.plot_corner()



